In [1]:
# Run in terminal or command prompt
# python3 -m spacy download en

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\ADMIN\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Import Dataset
fp = "F:datasets/pubnew.csv"
df = pd.read_csv(fp)
print(df.content.unique())
#print(df.target_names.unique())

['Matrix rank based ontology matching: An extension of string equality matchingIn this article, a string-based algorithm and the analogy of the total number of deviations that exist between two matrices are used for ontology matching. The type of matching and the distance between them are derived from the rank of the matrix. This process is useful for an agent programme to extract the knowledge from the application domain and to evaluate the result of matching to decide whether to include or exclude the ontology. As a case study to demonstrate the developed algorithm, the Fast Breeder Test Reactor (FBTR) is taken for analysis by representing its components, such as the steam generator system and the control rod drive mechanism, in formats such as RDF (Resource Description Framework) and OWL (Web Ontology Language).'
 'Sputtered AZO Thin Films for TCO and Back Reflector Applications in Improving the Efficiency of Thin Film a-Si:H Solar CellsWe report the properties of Al doped ZnO (AZO)

 'Round robin exercise on ball indentation technique in India: Indian nuclear reactor materials This paper presents outcome of round robin exercise on usage of ball indentation technique to predict strength properties of three different materials which are used in Indian nuclear reactors. The main objectives were to evolve a standardized procedure of determining the mechanical properties by the Ball Indentation technique and quantification of the variation in strength and fracture properties due to ageing through such tests. Standard uniaxial tensile tests were also carried out to quantify the differences between strength properties predicted by ball indentation and those determined using conventional tests. Three different materials studied were, carbon-manganese steel (Grade: SA 333Gr.6), stainless steel (SA312 Type 304LN) and zirconium alloy (Zr-2.5Nb). The carbon-manganese steel and stainless steel materials were drawn from extruded pipes while Zr-2.5Nb was drawn from pressure tube

In [5]:
df.head(15)

,content
0,Matrix rank based ontology matching: An extens...
1,Sputtered AZO Thin Films for TCO and Back Refl...
2,Assessment of natural radioactivity and radiol...
3,Accelerated rusting of reinforcing bars: The r...
4,Comparison of phase interaction models for hig...
5,Numerical simulation of Post Accident Heat Rem...
6,Transport phenomena in the electrodeionization...
7,Fixed bed adsorption of fluoride by Artocarpus...
8,Physicochemical properties of PUREX solvent on...
9,Kinetics of interphase transfer of zirconium b...


In [6]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-6-10af9153bd18>:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-6-10af9153bd18>:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


['Matrix rank based ontology matching: An extension of string equality '
 'matchingIn this article, a string-based algorithm and the analogy of the '
 'total number of deviations that exist between two matrices are used for '
 'ontology matching. The type of matching and the distance between them are '
 'derived from the rank of the matrix. This process is useful for an agent '
 'programme to extract the knowledge from the application domain and to '
 'evaluate the result of matching to decide whether to include or exclude the '
 'ontology. As a case study to demonstrate the developed algorithm, the Fast '
 'Breeder Test Reactor (FBTR) is taken for analysis by representing its '
 'components, such as the steam generator system and the control rod drive '
 'mechanism, in formats such as RDF (Resource Description Framework) and OWL '
 '(Web Ontology Language).']


In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['matrix', 'rank', 'based', 'ontology', 'matching', 'an', 'extension', 'of', 'string', 'equality', 'matchingin', 'this', 'article', 'string', 'based', 'algorithm', 'and', 'the', 'analogy', 'of', 'the', 'total', 'number', 'of', 'deviations', 'that', 'exist', 'between', 'two', 'matrices', 'are', 'used', 'for', 'ontology', 'matching', 'the', 'type', 'of', 'matching', 'and', 'the', 'distance', 'between', 'them', 'are', 'derived', 'from', 'the', 'rank', 'of', 'the', 'matrix', 'this', 'process', 'is', 'useful', 'for', 'an', 'agent', 'programme', 'to', 'extract', 'the', 'knowledge', 'from', 'the', 'application', 'domain', 'and', 'to', 'evaluate', 'the', 'result', 'of', 'matching', 'to', 'decide', 'whether', 'to', 'include', 'or', 'exclude', 'the', 'ontology', 'as', 'case', 'study', 'to', 'demonstrate', 'the', 'developed', 'algorithm', 'the', 'fast', 'breeder', 'test', 'reactor', 'fbtr', 'is', 'taken', 'for', 'analysis', 'by', 'representing', 'its', 'components', 'such', 'as', 'the', 'steam',

In [9]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

C:\Users\ADMIN\Anaconda3\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\Users\ADMIN\Anaconda3\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)


['matrix rank base ontology match extension string equality matchingin article string base algorithm analogy total number deviation that exist matrix be use ontology match type matching distance be derive rank matrix process be useful agent programme extract knowledge application domain evaluate result match decide include exclude ontology case study demonstrate developed algorithm fast breeder test reactor fbtr be take analysis represent  component such steam generator system control rod drive mechanism format such rdf resource description framework owl web ontology language', 'sputter azo thin film tco back reflector application improve efficiency thin film si solar cellswe report property dope zno azo thin film glass substrate  effect efficiency amorphous silicon solar cell back reflector orient azo thin film be grow use magnetron sputtering vary gas flow rat influence flow rate structural electrical optical property azo thin film suitable transparent conduct oxide tco back reflecto

In [10]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [11]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  6.277618364418938 %


In [12]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_topics=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

C:\Users\ADMIN\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, n_topics=20, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


In [13]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -169839.20190470692
Perplexity:  450.52916152996573
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'n_topics': 20,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [14]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

C:\Users\ADMIN\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\ADMIN\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 10}
Best Log Likelihood Score:  -66746.78178801028
Model Perplexity:  412.073248374748


In [30]:
# Get Log Likelyhoods from Grid Search Output
n_topics = [10, 15, 20, 25, 30]
log_likelyhoods_5 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.5]
log_likelyhoods_7 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.7]
log_likelyhoods_9 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.9]

# Show graph
plt.figure(figsize=(12, 8))
plt.plot(n_topics, log_likelyhoods_5, label='0.5')
plt.plot(n_topics, log_likelyhoods_7, label='0.7')
plt.plot(n_topics, log_likelyhoods_9, label='0.9')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [28]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_topics)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

TypeError: 'NoneType' object cannot be interpreted as an integer

In [27]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)


In [28]:
# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_topics)]

TypeError: 'NoneType' object cannot be interpreted as an integer